In [ ]:
# 환경변수 가져오기
# openai_api_key = os.getenv("OPENAI_API_KEY")
# serpapi_key = os.getenv("SERPAPI_API_KEY")

# 또는 다음과 같이 직접 키 입력 (개발)
# os.environ["OPENAI_API_KEY"] = ""  # 자신의 OpenAI 키
# os.environ["SERPAPI_API_KEY"] = ""

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

#### 다음 실습 코드는 학습 목적으로만 사용 바랍니다. 문의 : audit@korea.ac.kr 임성열 Ph.D.

#### LangGraph 기반 LangChain Agent 구현
이 노트북은 다음의 Tool들을 사용하는 LangGraph Agent를 구현하고 실행합니다:

1. **사칙연산 함수**
2. **PDF 업로드 및 임베딩 후 FAISS 저장**
3. **웹 크롤링을 통한 트렌드 저장**
4. **벡터 DB로부터 요약 및 등장 횟수 카운트**

In [ ]:
# 필요한 패키지 설치 (필요 시 주석 제거)
%pip install langchain langgraph faiss-cpu openai PyMuPDF beautifulsoup4 requests

In [2]:
import os
from langchain.tools import tool
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.document_loaders import PyMuPDFLoader
from langchain.chains import RetrievalQA
import requests
from bs4 import BeautifulSoup
from collections import Counter
import re

#### 1. 사칙연산 함수 정의

In [3]:
@tool
def calculate(expression: str) -> str:
    """사칙연산을 수행한다. 예: '2 + 3 * 4'"""
    try:
        result = eval(expression)
        return f"{expression} = {result}"
    except Exception as e:
        return f"오류: {str(e)}"

#### 2. PDF 임베딩 및 벡터 저장

In [4]:
import os
from langchain_core.tools import tool
from langchain.document_loaders import PyMuPDFLoader
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 전역 벡터 DB
pdf_vectorstore = None

@tool
def embed_pdf_to_faiss(path: str) -> str:
    """
    PDF 파일의 상대 경로 또는 절대 경로를 받아 절대 경로로 변환한 후,
    해당 PDF를 임베딩하여 벡터 DB에 저장합니다.
    """
    global pdf_vectorstore

    # 절대 경로로 변환
    abs_path = os.path.abspath(path)

    if not os.path.exists(abs_path):
        return f"❌ PDF 파일이 존재하지 않습니다: {abs_path}"

    loader = PyMuPDFLoader(abs_path)
    documents = loader.load()

    embeddings = OpenAIEmbeddings()
    pdf_vectorstore = FAISS.from_documents(documents, embeddings)

    return f"✅ '{abs_path}' PDF가 벡터 DB에 저장되었습니다."



#### 3. 네이버 트렌드 크롤링 및 저장

In [5]:
# def crawl_trend_and_store(keyword: str) -> str:
#     """네이버 검색 트렌드를 크롤링하고 벡터 DB에 저장한다."""
#     global web_vectorstore
#     url = f"https://search.naver.com/search.naver?query={keyword}"
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, 'html.parser')
#     paragraphs = [p.get_text() for p in soup.find_all('p')]
#     contents = [keyword + ":\n" + p for p in paragraphs if keyword.lower() in p.lower()]
#     documents = [Document(page_content=c) for c in contents]
#     embeddings = OpenAIEmbeddings()
#     web_vectorstore = FAISS.from_documents(documents, embeddings)
#     return f"✅ '{keyword}' 검색결과가 벡터 DB에 저장되었습니다."

# 3️⃣ 네이버 검색 크롤링 + 임베딩
@tool
def crawl_naver_and_embed(term: str) -> str:
    """
    네이버 뉴스 API를 사용해 특정 키워드로 뉴스 기사를 검색하고,
    그 결과를 임베딩하여 FAISS 벡터 DB에 저장합니다.
    """
    global web_vectorstore

    headers = {
        "X-Naver-Client-Id": "gbqzUVViEiF6WXhuq3gZ",
        "X-Naver-Client-Secret": "y0YXaa5unU"
    }

    url = "https://openapi.naver.com/v1/search/news.json"
    params = {
        "query": term,
        "display": 5,
        "sort": "date"
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
    except requests.RequestException as e:
        return f"❌ 네이버 API 호출 실패: {e}"

    data = response.json()
    items = data.get("items", [])

    if not items:
        return "🔍 검색 결과가 없습니다."

    documents = []
    for item in items:
        title = re.sub(r"<.*?>", "", item.get("title", ""))
        description = re.sub(r"<.*?>", "", item.get("description", ""))
        content = f"{title}\n{description}"
        documents.append(Document(page_content=content, metadata={"source": "NaverNews"}))

    splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
    docs = splitter.split_documents(documents)

    embeddings = OpenAIEmbeddings()
    web_vectorstore = FAISS.from_documents(docs, embeddings)

    return f"✅ '{term}'에 대한 뉴스 트렌드가 벡터 DB에 저장되었습니다."



#### 4. PDF와 웹 벡터 DB 요약 및 빈도수 분석

In [6]:
@tool
def summarize_and_count(keyword: str) -> str:
    """PDF와 웹에서 해당 키워드를 요약하고 등장 횟수를 계산"""
    pdf_docs = pdf_vectorstore.similarity_search(keyword, k=10)
    web_docs = web_vectorstore.similarity_search(keyword, k=10)

    pdf_text = "\n".join([doc.page_content for doc in pdf_docs])
    web_text = "\n".join([doc.page_content for doc in web_docs])

    def count_keyword(text):
        return len(re.findall(keyword, text, re.IGNORECASE))

    count_pdf = count_keyword(pdf_text)
    count_web = count_keyword(web_text)

    return f"📘 PDF 요약: {pdf_text[:300]}...\n🕸 웹 트렌드 요약: {web_text[:300]}...\n'📊 등장 횟수 - PDF: {count_pdf}회, 웹: {count_web}회"

In [7]:
import os
print("현재 작업 디렉토리:", os.getcwd())
print("attention.pdf 존재 여부:", os.path.exists("attention.pdf"))

현재 작업 디렉토리: /Users/phoenix/Eagle/2025_LangChain/langchain_lab
attention.pdf 존재 여부: True


#### 5. LangGraph Agent 실행

In [ ]:
from langchain.agents import Tool, AgentExecutor, initialize_agent
from langchain.agents.agent_types import AgentType
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
#LangChain 최신 버전에서는 text splitter들이 langchain_text_splitter*라는 별도 모듈로 분리됨


# 1️⃣ 함수들을 @tool로 정의 (또는 Tool(description=..., func=...) 형태로 래핑)
@tool
def calculate(expression: str) -> str:
    """수식을 계산합니다. 예: '3+4*2'"""
    try:
        result = eval(expression)
        return f"계산 결과는 {result}입니다."
    except Exception as e:
        return f"계산 오류: {e}"

# 다른 tool들도 동일하게 정의되어 있다고 가정 (embed_pdf_to_faiss, crawl_naver_and_embed, summarize_and_count)

# 2️⃣ Tool 리스트 정의
tools = [calculate, embed_pdf_to_faiss, crawl_naver_and_embed, summarize_and_count]
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# 3️⃣ LLM 설정 (예: gpt-4)
llm = ChatOpenAI(model="gpt-4", temperature=0)

# 4️⃣ AgentExecutor 정의 : 실제로는 Agent가 단순히 Tool Calling만 하지는 않는다.
# Agent 역할에 맞는 AI 서비스 처리를 한 결과를 가지고 다음 노드로 분기하거나 이어지며, 필요 시 Tool Calllin을 하게 된다.
# LangChain을 이용한 Agent는, Tool Calling으로 세부 사항을 처리하거나 이전 노드의 출력(State)을 다음 노드에서 입력으로 받아 처리하도록 설계한다.
executor = initialize_agent(
    agent=AgentType.OPENAI_FUNCTIONS,
    tools=tools,
    llm=llm,
    verbose=True
)

# 5️⃣ 실행 (함수명 말고 자연어 입력으로 실행)
print(executor.invoke({"input": "다음 PDF 파일을 벡터 DB에 저장하라. 파일 경로는 './attention.pdf' 이다."}))
print(executor.invoke({"input": "GPU 라는 키워드로 뉴스 트렌드를 검색하라."}))
print(executor.invoke({"input": "GPU 라는 키워드에 대해 PDF와 웹을 요약하고 자주 나온 단어를 제시하라."}))


/var/folders/xr/0xbgl9wd3wz914nd58ylmykw0000gp/T/ipykernel_78154/2527526929.py:30: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  executor = initialize_agent(




> Entering new AgentExecutor chain...

Invoking: `embed_pdf_to_faiss` with `{'path': './attention.pdf'}`


✅ '/Users/phoenix/Eagle/2025_LangChain/langchain_lab/attention.pdf' PDF가 벡터 DB에 저장되었습니다.'./attention.pdf' 파일이 벡터 DB에 성공적으로 저장되었습니다. 다른 도움이 필요하시면 알려주세요.

> Finished chain.
{'input': "다음 PDF 파일을 벡터 DB에 저장하라. 파일 경로는 './attention.pdf' 이다.", 'output': "'./attention.pdf' 파일이 벡터 DB에 성공적으로 저장되었습니다. 다른 도움이 필요하시면 알려주세요."}


> Entering new AgentExecutor chain...

Invoking: `crawl_naver_and_embed` with `{'term': 'GPU'}`


✅ 'GPU'에 대한 뉴스 트렌드가 벡터 DB에 저장되었습니다.'GPU'에 대한 뉴스 트렌드를 성공적으로 검색하고 벡터 DB에 저장했습니다. 다른 도움이 필요하시면 알려주세요.

> Finished chain.
{'input': 'GPU 라는 키워드로 뉴스 트렌드를 검색하라.', 'output': "'GPU'에 대한 뉴스 트렌드를 성공적으로 검색하고 벡터 DB에 저장했습니다. 다른 도움이 필요하시면 알려주세요."}


> Entering new AgentExecutor chain...

Invoking: `summarize_and_count` with `{'keyword': 'GPU'}`


📘 PDF 요약: 1
Introduction
Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural networks
in particular, have be